In [1]:
import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [22]:
impression_embed_size = 50 # 853,540개 
filter_embed_size = 10   # 208개
platform_embed_size = 5       # 55개
city_embed_size = 20          # 34,752개
device_embed_size = 3 # mobile, desktop, tablet one-hot

rnn_input_size = impression_embed_size # action 5개로 통합 후 one-hot 

#LSTM
hidden_state_size = 64

n_hidden_1 = 128
n_hidden_2 = 128
n_hidden_3 =64

In [28]:
# Parameters
learning_rate = 0.001
seq_length = 20
num_epoch = 10
batch_size = 1

In [4]:
import pandas as pd
import functions as f
from collections import Counter

In [5]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
meta_df = pd.read_csv('./item_metadata.csv')
submit_df = pd.read_csv('./submission_popular.csv')

popular_df = f.get_popularity(train_df)

In [6]:
poi_idx = np.load('./npy/poi_names.npy', allow_pickle=True)
poi_idx = list(poi_idx)
impressions_idx = np.load('./npy/impressions_index.npy', allow_pickle=True)
impressions_idx = list(impressions_idx)
city_idx = np.load('./npy/city_names.npy', allow_pickle=True)
city_idx = list(city_idx)
platform_idx = np.load('./npy/platform_names.npy', allow_pickle=True)
platform_idx = list(platform_idx)
filter_idx = np.load('./npy/filter_merged.npy', allow_pickle=True)
filter_idx = list(filter_idx)
action_idx = np.load('./npy/action_type_names.npy', allow_pickle=True)
action_idx = list(action_idx)

sort_order_idx = np.load('./npy/sorting_names.npy', allow_pickle=True)
sort_order_idx = list(sort_order_idx)
device_idx = np.load('./npy/device_names.npy', allow_pickle=True)
device_idx = list(device_idx)

# impression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])
# city_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])
# platform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])
# filter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])

In [7]:
train_label = train_df[(train_df["action_type"] == "clickout item")]     #train_label : (1,586,586, 12)
batch_indexes = np.random.permutation(len(train_label))

In [8]:
def recurrent_neural_network_model(impression_list):
    
    embed = tf.nn.embedding_lookup(impression_embedding, impression_list)  # batch x seq_length x embedding size
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_state_size)    
    _ , states = tf.nn.dynamic_rnn(lstm_cell, embed, dtype=tf.float32)   
    
    return states

In [9]:
def getImpressionList(user_label):
    step = user_label['step']
    if step > seq_length+1:
        step = seq_length+1
    label_point = user_label.name
    history_df = train_df[label_point-step+1:label_point]
    
    impress_df = history_df[(history_df['action_type']==action_idx[1])|(history_df['action_type']==action_idx[3])|(history_df['action_type']==action_idx[4])\
                           |(history_df['action_type']==action_idx[5])|(history_df['action_type']==action_idx[6])|(history_df['action_type']==action_idx[8])]
    
    impression_list = np.array(impress_df['reference']).astype(int)
        
    if len(impression_list) < seq_length:
        impression_list = np.concatenate((np.zeros([seq_length-len(impression_list)], dtype=int), impression_list))
    
    return np.reshape(impression_list, [-1, seq_length])

In [26]:
# Define the neural network
def dense_net(context, train_label, target_impression):   
    
    context = tf.reshape(context, [64])
        #platform
    platform_embed = embedding_function('platform', train_label[-1,INDEX_platform])
    
    #city, search for destination 해서 검색해봤던 city들
    city_embed = embedding_function('city', train_label[-1,INDEX_city])
    
    #device
    device_embed = embedding_function('device', train_label[-1,INDEX_device])
    
    #current_filter
    
    filter_b = train_label[-1,INDEX_current_filters]
    if pd.isna(filter_b):
        final_embed_current_filter = tf.zeros(filter_embed_size)
    else:
        final_embed_current_filter = tf.zeros(filter_embed_size)
#         filters_b = f.string_to_array(filter_b)
#         final_embed_current_filter = getWeightedAverage('filter', filters_b , filter_embed_size)
    
#     print("context", np.shape(context))
#     print("platform_embed", np.shape(platform_embed))
#     print("city_embed", np.shape(city_embed))
#     print("device_embed", np.shape(device_embed))
    
    XX_input= tf.concat((context, platform_embed, city_embed, device_embed),axis=0)
    XX_input = tf.reshape(XX_input, [batch_size, 122])
    layer_1 = tf.layers.dense(XX_input, n_hidden_1,activation=tf.nn.relu)
    layer_2 = tf.layers.dense(layer_1, n_hidden_2,activation=tf.nn.relu)
    layer_3 = tf.layers.dense(layer_2, n_hidden_3, activation=tf.nn.relu)
    out_layer = tf.layers.dense(layer_3, 1, activation=tf.nn.sigmoid)
    return out_layer

In [20]:
INDEX_user_id =0                                                 
INDEX_session_id =1                                            
INDEX_timestamp =2                                                
INDEX_step        =3                                                       
INDEX_action_type =4                                       
INDEX_reference   =5                                               
INDEX_platform    =6                                                
INDEX_city        =7                                          
INDEX_device      =8                                                 
INDEX_current_filters=9                                                
INDEX_impressions    =10   
INDEX_prices         =11   

In [18]:
def embedding_function(domain, key):
    
    embedded_vector = tf.zeros([impression_embed_size])
    if domain == action_idx[9]:  #'poi'
        if key in poi_idx:
            idx = poi_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(poi_embedding, idx)
        
    elif (domain == action_idx[1])|(domain == action_idx[3])|(domain == action_idx[4])|(domain == action_idx[5])\
    |(domain == action_idx[6])|(domain == action_idx[8]):  #impression
        if key in impressions_idx:
            idx = impressions_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(impression_embedding, idx)
    
    elif domain == action_idx[7]: # search for destination, 'city'
        if key in city_idx:
            idx = city_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(city_embedding, idx)

    elif domain == 'platform':
        if key in platform_idx:
            idx = platform_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(platform_embedding, idx)
        
    elif domain ==action_idx[2]:  #'filter'
        if key in filter_idx:
            idx = filter_idx.index(key)
        else:
            idx = -1
        embedded_vector = tf.nn.embedding_lookup(filter_embedding, idx)
        
    elif domain ==action_idx[0]:  #sorting
        embedded_vector = np.zeros([sort_embed_size])
        embedded_vector[sort_order_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    elif domain =='device':
        embedded_vector = np.zeros([device_embed_size])
#         embedded_vector[device_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    return embedded_vector

In [14]:
# Define the neural network
tf.reset_default_graph()
# Act_list = tf.placeholder(tf.string, [None, seq_length])
Impression_list = tf.placeholder(tf.int32, [None, seq_length])

Train_Label_df = tf.placeholder(tf.string, [None, 12])
Target_impression = tf.placeholder(tf.int32, [None, 1])
Y_label = tf.placeholder(tf.int32, [None, 1])

# poi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])
impression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])
city_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])
platform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])
filter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])

In [15]:
context = recurrent_neural_network_model(Impression_list)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [29]:
output = dense_net(context[0], Train_Label_df, Target_impression)

context (64,)
platform_embed (5,)
city_embed (50,)
device_embed (3,)


In [30]:
loss = tf.losses.sigmoid_cross_entropy(Y_label, output)

Instructions for updating:
Use tf.cast instead.


In [31]:
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:
Use tf.cast instead.


In [33]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())

In [34]:
label_df = None
for k in range(2):
    idx = batch_indexes[k*batch_size:(k+1)*batch_size]
    label_df = train_label.iloc[idx]
    for i in range(2):  # batch size
        user_label = label_df.iloc[i]
        train_label_batch = user_label["reference"]  # 클릭한 reference number
        train_candidates = user_label['impressions']
        train_candidates = f.string_to_array(train_candidates)   
        
        candidate_prices = user_label['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)
        
        label_index = train_candidates.index(train_label_batch)
        labels = np.zeros([len(train_candidates)],dtype=int)
        labels[label_index]=1
        
#         act_list, ref_list = getActionList(user_label, seq_length)
        impression_list = getImpressionList(user_label)
        user_label = user_label.fillna('Nope')
        users = np.array(user_label)
        users= np.reshape(users,[-1,12])     
        
        for h in range(len(train_candidates)):
            _, loss_val = sess.run([train, loss], feed_dict={Impression_list: impression_list, Train_Label_df: users,\
                                                             Y_label:np.reshape(labels[h],[-1,1]), \
                                                             Target_impression: np.reshape(int(train_candidates[h]),[-1,1])})
            print("loss : ", loss_val)

InternalError: Unable to get element as bytes.